In [1]:
import findspark
findspark.init('D:\spark-2.3.3-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
print('Modules Imported')

Modules Imported


In [2]:
spark=SparkSession.builder.appName('cruise').getOrCreate()

In [55]:
df=spark.read.csv('D:\python coding\pyspark_tutorial\Linear regression\cruise_ship_info.csv'
                  ,inferSchema=True,header=True)
df.show(10)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [8]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [9]:
for item in df.head(5):
    print(item)
    print('\n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)




In [50]:
df.groupBy('Cruise_line').count().show()
df.columns

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [12]:
#to convert string into a numeric feature use stringinddexer
from pyspark.ml.feature import StringIndexer

In [54]:
indexer=StringIndexer(inputCol='Cruise_line',outputCol='cruise_cat')
indexed=indexer.fit(df).transform(df)

for item in indexed.head(5):
    print(item)
    print('\n')
#indexed.select('Cruise_line','cruise_cat').show()

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0)




In [18]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


In [21]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [57]:
#creating vectors now
assembler=VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'cruise_cat'],outputCol='features')
output=assembler.transform(indexed)

output.select('features','crew').show(5)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
+--------------------+----+
only showing top 5 rows



In [28]:
final_data=output.select('features','crew')

In [29]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [30]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              110|
|   mean|8.036545454545466|
| stddev|3.494682050706418|
|    min|              0.6|
|    max|             21.0|
+-------+-----------------+



In [58]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                48|
|   mean|7.2387500000000005|
| stddev| 3.496559174936095|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+



In [31]:
from pyspark.ml.regression import LinearRegression

In [32]:
ship_lr=LinearRegression(featuresCol='features',labelCol='crew')

In [33]:
trained_ship_model=ship_lr.fit(train_data)

In [59]:
ship_results=trained_ship_model.evaluate(train_data)

In [62]:
print('Mean Square Error :',ship_results.rootMeanSquaredError)

Mean Square Error : 0.9339423911007266


In [63]:
print('Rsquared Error :',ship_results.r2)

Rsquared Error : 0.9279239421964878


In [40]:
from pyspark.sql.functions import corr

In [45]:
#checking the correlation between columns
#correlation is always between tow columns
df.select(corr('crew','passengers')).show()


+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [60]:
#testing model on unlabeled data
unlabeled_data=test_data.select('features')
unlabeled_data.show(5)

+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,93.0,23.94,9...|
|[6.0,158.0,43.7,1...|
|[7.0,89.6,25.5,9....|
|[9.0,90.09,25.01,...|
+--------------------+
only showing top 5 rows



In [61]:
predictions=trained_ship_model.transform(unlabeled_data)
predictions.show(10)

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[6.0,30.276999999...| 4.290267415583632|
|[6.0,93.0,23.94,9...|10.561152805966858|
|[6.0,158.0,43.7,1...|14.018897542771231|
|[7.0,89.6,25.5,9....|11.051686362634006|
|[9.0,90.09,25.01,...| 9.355005994457068|
|[9.0,113.0,26.74,...|11.363942765011508|
|[10.0,58.825,15.6...| 7.295815705089378|
|[10.0,81.76899999...| 8.805872457623725|
|[10.0,90.09,25.01...| 8.981103875979219|
|[10.0,105.0,27.2,...|11.252031636345482|
+--------------------+------------------+
only showing top 10 rows

